In [57]:
import ee
import pandas as pd

ee.Initialize()

# 관심 지역 및 기간 설정
aoi = ee.Geometry.Polygon(
    [[[
                            128.852806,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.044895
                        ],
                        [
                            128.940868,
                            35.003003
                        ],
                        [
                            128.852806,
                            35.003003
                        ]]]
)

start_date = '2021-11-01'
end_date = '2023-10-31'

# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    # Selecting vand.
    red = image.select('B4')   # Red vand
    nir = image.select('B8')   # NIR vand
    swir1 = image.select('B11') # SWIR1 vand
    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# 조건에 맞는 픽셀 수 계산
def count_pixels(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    masked = image.updateMask(image.select('NDWI').gt(0.5))
    fai_count = masked.select('FAI').gt(-5).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10
    ).get('FAI')
    return ee.Feature(None, {'date': date, 'fai_count': fai_count})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용 후 픽셀 수 계산
processed_collection = sentinel2.map(calculate_indices)
pixel_counts = processed_collection.map(count_pixels)

# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
pixel_count_features = pixel_counts.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in pixel_count_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')


In [58]:
df.describe()

,fai_count
count,60.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [59]:
# Calculate the area of the AOI in square meters.
area = aoi.area().getInfo()

# Convert the area to square kilometers (1 square kilometer = 1,000,000 square meters).
area_sq_km = area / 1_000_000

print("Area of the AOI in square kilometers:", area_sq_km)

Area of the AOI in square kilometers: 37.353208256071845


In [60]:
# Sentinel-2 ImageCollection 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# NDWI와 FAI 계산
def calculate_indices(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lambda_nir = 832.8
    lambda_red = 664.6
    lambda_swir1 = 1613.7

    red = image.select('B4')   # Red band
    nir = image.select('B8')   # NIR band
    swir1 = image.select('B11') # SWIR1 band

    # Calculate FAI
    fai = nir.subtract(red).add(
        swir1.subtract(red).multiply(
            (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red)
        )
    ).rename('FAI')
    return image.addBands([ndwi, fai])

# NDWI가 0.5 이상이고 FAI가 50 이상인 지역의 면적 계산
def calculate_area(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mask = image.select('NDWI').gt(0.1).And(image.select('FAI').gt(-50))
    area = mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10  # Sentinel-2 픽셀 해상도
    ).get('NDWI')
    return ee.Feature(None, {'date': date, 'area': area})

# 이미지 컬렉션에 NDWI와 FAI 계산 적용 후 면적 계산
processed_collection = sentinel2.map(calculate_indices)
area_series = processed_collection.map(calculate_area)

# 결과를 서버 측 객체로 변환 (Python 클라이언트로 가져오기)
area_features = area_series.getInfo()['features']

# 결과를 pandas DataFrame으로 변환
df = pd.DataFrame([feat['properties'] for feat in area_features])

# DataFrame을 'date' 컬럼에 따라 오름차순으로 정렬
df = df.sort_values(by='date')


In [61]:
df.describe()

,area
count,5.600000e+01
mean,5.371852e+06
std,5.954240e+06
min,0.000000e+00
25%,1.106464e+06
50%,2.998594e+06
75%,7.304144e+06
max,2.078645e+07


In [62]:
# Sentinel-2 ImageCollection 필터링
sentinel2_image = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .first()

In [63]:
# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['WEVI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']  # 색상 팔레트 설정
}

In [ ]:
import geemap

In [ ]:

# 가장 최근의 이미지 선택
latest_image = sentinel2.sort('system:time_start', False).first()

# NDWI 계산
ndwi = latest_image.normalizedDifference(['B3', 'B8']).rename('NDWI')

# 수역 마스크 생성 (NDWI 임계값 설정)
water_mask = ndwi.lte(0.1)

# 마스크 적용하여 수역만 추출
masked_image = latest_image.updateMask(water_mask)

# geemap Map 객체 생성
Map = geemap.Map()
Map.centerObject(aoi, 10)  # 관심 지역을 중심으로 설정

# 지도에 이미지 추가
Map.addLayer(masked_image, {'bands': ['B4', 'B3', 'B2'], 'max': 2000}, 'Water Masked')

# 지도 표시
Map

Map(center=[35.0239551627125, 128.8968370000004], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
# 가장 최근의 이미지 선택 및 NDWI와 FAI 계산
latest_image = sentinel2.sort('system:time_start', False).first()
processed_image = calculate_indices(latest_image)

# FAI가 0.3 이하인 지역 마스크 생성
water_mask1 = processed_image.select('FAI').gte(0)

# 마스크 적용하여 지정된 조건에 맞는 지역만 추출
masked_image1 = processed_image.updateMask(water_mask1)

# geemap Map 객체 생성
Map = geemap.Map()
Map.centerObject(aoi, 10)  # 관심 지역을 중심으로 설정

# 지도에 이미지 추가
Map.addLayer(masked_image1, {'bands': ['B4', 'B3', 'B2'], 'max': 2000}, 'FAI Masked')

# 지도 표시
Map

Map(center=[35.0239551627125, 128.8968370000004], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
# 지도에 이미지 추가
Map.addLayer(masked_image, {'bands': ['B4', 'B3', 'B2'], 'max': 2000}, 'Water Masked')

# 지도 표시
Map

In [78]:
# geemap Map 객체 생성
Map = geemap.Map()
Map.centerObject(aoi, 10)  # 관심 지역을 중심으로 설정

# FAI 지수 시각화를 위한 파라미터 설정
fai_params = {
    'min': -500, 'max': 25, 'palette': ['blue', 'white', 'green']
}

# 지도에 FAI 지수 레이어 추가
Map.addLayer(processed_image.select('FAI'), fai_params, 'FAI')

# 지도 표시
Map

Map(center=[35.0239551627125, 128.8968370000004], controls=(WidgetControl(options=['position', 'transparent_bg…